# SIEM Use Cases

- What IP address (and countries) are attacking the WAN endpoint?
 - Visualise those against global map 
 - Block non AU countries

- Validate the firewall rules for VLAN isolation:
 - Flow from VLAN x to y is valid/not valid
 - Alert on not valid flows 

- Identify Unifi component anomalies
 - PAM activity ({job="ubnt-authpriv"} requires review)

- Detect loss of internet connectivity

In [64]:
import requests
import json
import pandas as pd
from datetime import datetime, timedelta

# default to last 3 hours
dend = int(datetime.utcnow().timestamp())*1000000000
dstart = datetime.now() - timedelta(hours=3)
dstart = int(dstart.timestamp()*1000000000)

url = "http://loki.loki:3100/loki/api/v1/query_range"
headers = {'Content-type': 'application/json'}
params = {
    'query': '{job="ubnt-geo"}', # logs that contain external actors attempting to ingress via WAN
    'direction': 'BACKWARD',
    'limit': 100,
    'start': dstart,
    'end': dend,
    'step': 10
}

resp = requests.get(url, headers = headers, params = params)
if (resp.status_code != 200):
    print("Failed to get data")
json_obj = (json.loads(resp.text))

# Refer https://github.com
#   /BindiChen/machine-learning/blob/main/data-analysis/027-pandas-convert-json/pandas-convert-json.ipynb
df = (pd.json_normalize(
    json_obj, 
    record_path = ['data','result','values'])
)
df


,0,1
0,1651054143000000000,"{""geoplugin_request"":""17.133.234.33"",""geoplugi..."
1,1651054141000000000,"{""geoplugin_request"":""17.133.234.33"",""geoplugi..."
2,1651054141000000000,"{""geoplugin_request"":""17.133.234.33"",""geoplugi..."
3,1651054127000000000,"{""geoplugin_request"":""31.43.191.143"",""geoplugi..."
4,1651054126000000000,"{""geoplugin_request"":""31.43.191.143"",""geoplugi..."
...,...,...
95,1651053382000000000,"{""geoplugin_request"":""179.43.167.122"",""geoplug..."
96,1651053379000000000,"{""geoplugin_request"":""159.223.233.224"",""geoplu..."
97,1651053375000000000,"{""geoplugin_request"":""31.43.191.143"",""geoplugi..."
98,1651053373000000000,"{""geoplugin_request"":""103.145.4.138"",""geoplugi..."


In [66]:
from glom import glom
import re

# Find IPs in syslog msg: SRC and DEST. 
# We assume SRC is the first regex group (0).
IP4_REGEX = r'(?:[0-9]{1,3}\.){3}[0-9]{1,3}'
df1 = df[1].apply(lambda row: 
    re.search( 
        IP4_REGEX,
        glom(json.loads(row), 'geoplugin_request')
    ).group(0)
)
df1


0       17.133.234.33
1       17.133.234.33
2       17.133.234.33
3       31.43.191.143
4       31.43.191.143
           ...       
95     179.43.167.122
96    159.223.233.224
97      31.43.191.143
98      103.145.4.138
99     45.155.204.166
Name: 1, Length: 100, dtype: object

In [17]:
# Basemap library
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
 
# Set the dimension of the figure
plt.rcParams["figure.figsize"]=15,10;

# Make the background map
m=Basemap(llcrnrlon=-180, llcrnrlat=-65, urcrnrlon=180, urcrnrlat=80, projection='merc');
m.drawmapboundary(fill_color='#A6CAE0', linewidth=0);
m.fillcontinents(color='grey', alpha=0.3);
m.drawcoastlines(linewidth=0.1, color="white");

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 96 from C header, got 80 from PyObject